In [12]:
import json
from llama_index.core import Document, VectorStoreIndex
#from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#from llama_index.core.node_parser import SimpleNodeParser
#from llama_index.core.storage import StorageContext
#from transformers import pipeline
#from langchain.chains import RetrievalQA
#from langchain.llms import HuggingFacePipeline
from llama_index.core import StorageContext, load_index_from_storage

In [13]:
#import google.generativeai as genai
#from langchain.chains import RetrievalQA



In [14]:


# Charger le fichier JSON
with open("documents.json", "r") as file:
    data = json.load(file)

# Vérification
print(f"Nombre de chunks : {len(data)}")
print(data[0])  # Affiche le premier chunk pour vérifier la structure


Nombre de chunks : 1542
{'content': 'le gabon a mené plusieurs actions ces dernières années pour valoriser la transformation industrielle de son bois, volet porteur d’une économie jusqu’ici dominée par le pétrole. l’indice composite des activités industrielles du domaine du bois au gabon a grimpé de 4% en glissement trimestriel au 3e trimestre 2024, selon la dernière note conjoncturelle du ministère de l’economie.', 'metadata': {'titre': 'Gabon la production industrielle de bois en hausse de 4 au 3e trimestre 2024', 'date': '25/01/2025', 'url': 'https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024'}}


In [15]:
# Convertir les chunks en Documents
documents = [
    Document(
        text=chunk["content"],
        metadata=chunk["metadata"]
    )
    for chunk in data
]


In [20]:
# Créer le modèle Hugging Face Embedding
embedding = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")


# Créer un index vectoriel avec les embeddings locaux
index = VectorStoreIndex.from_documents(
    documents=documents,
    embed_model=embedding
)

# Sauvegarder l'index
storage_context = StorageContext.from_defaults()
index.storage_context.persist(persist_dir="Save/index_data")
print("Index vectoriel créé et sauvegardé avec succès !")




Index vectoriel créé et sauvegardé avec succès !


In [21]:
#Configurer l'embedding local
embedding = HuggingFaceEmbedding(model_name="all-MiniLM-L6-v2")

#Charger l'index vectoriel en spécifiant l'embedding
storage_context = StorageContext.from_defaults(persist_dir="Save/index_data")
index = load_index_from_storage(storage_context, embed_model=embedding)

print("Index vectoriel chargé avec succès!")


Index vectoriel chargé avec succès!


In [22]:
retriever = index.as_retriever()
query = "Quelles sont les actions du Gabon dans le secteur du bois ?"
retrieved_docs = retriever.retrieve(query)

for doc in retrieved_docs:
    print(f" Document : {doc.text[:200]}...") 
    print(f" Métadonnées : {doc.metadata}")


 Document : alors que brice oligui n’guema (photo) qui dirige le pays depuis le putsch d’août 2023 avait promis dès son arrivée de rendre le pouvoir aux civils à l’issue de la transition, les ambitions politiques...
 Métadonnées : {'titre': 'Gabon la presidentielle aura lieu le 12 avril gouvernement', 'date': '23/01/2025', 'url': 'https://www.agenceecofin.com/actualites/2301-125195-gabon-la-presidentielle-aura-lieu-le-12-avril-gouvernement'}
 Document : l’indice pour le bois de placage a quant à lui grimpé de 11% par rapport au trimestre précédent, soutenu par la demande en provenance de chine. il n’y a néanmoins pas eu d’embellie sur le segment du s...
 Métadonnées : {'titre': 'Gabon la production industrielle de bois en hausse de 4 au 3e trimestre 2024', 'date': '25/01/2025', 'url': 'https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024'}


In [23]:
retriever = index.as_retriever()
query = "Quelles sont les rapports entre la France et le Gabon ?"
retrieved_docs = retriever.retrieve(query)

for doc in retrieved_docs:
    print(f" Document : {doc.text[:200]}...") 
    print(f" Métadonnées : {doc.metadata}")


 Document : un partenariat avec la france pourrait permettre au gabon de consolider sa position de leader dans le domaine du numérique en afrique centrale. ce partenariat vise à accroître la contribution du secte...
 Métadonnées : {'titre': 'Numerique la france explore des opportunites dinvestissement au gabon', 'date': '20/01/2025', 'url': 'https://www.agenceecofin.com/actualites-numerique/2001-125057-numerique-la-france-explore-des-opportunites-dinvestissement-au-gabon'}
 Document : le gabon se positionne comme un acteur majeur du numérique en afrique centrale. pour continuer à développer ce secteur, le pays mise sur l’attraction des investissements étrangers....
 Métadonnées : {'titre': 'Numerique la france explore des opportunites dinvestissement au gabon', 'date': '20/01/2025', 'url': 'https://www.agenceecofin.com/actualites-numerique/2001-125057-numerique-la-france-explore-des-opportunites-dinvestissement-au-gabon'}


In [24]:
models = ["all-MiniLM-L6-v2", "all-mpnet-base-v2", "all-distilroberta-v1"]
query = "Quelles sont les actions du Gabon dans le secteur du bois ?"

for model_name in models:
    print(f"\n🔹 Test avec le modèle {model_name}")
    embedding = HuggingFaceEmbedding(model_name=model_name)

    retriever = index.as_retriever()  # Utiliser le même index
    retrieved_docs = retriever.retrieve(query)

    for doc in retrieved_docs:
        print(f"📄 {doc.text[:200]}... | 🔗 {doc.metadata}")



🔹 Test avec le modèle all-MiniLM-L6-v2
📄 alors que brice oligui n’guema (photo) qui dirige le pays depuis le putsch d’août 2023 avait promis dès son arrivée de rendre le pouvoir aux civils à l’issue de la transition, les ambitions politiques... | 🔗 {'titre': 'Gabon la presidentielle aura lieu le 12 avril gouvernement', 'date': '23/01/2025', 'url': 'https://www.agenceecofin.com/actualites/2301-125195-gabon-la-presidentielle-aura-lieu-le-12-avril-gouvernement'}
📄 l’indice pour le bois de placage a quant à lui grimpé de 11% par rapport au trimestre précédent, soutenu par la demande en provenance de chine. il n’y a néanmoins pas eu d’embellie sur le segment du s... | 🔗 {'titre': 'Gabon la production industrielle de bois en hausse de 4 au 3e trimestre 2024', 'date': '25/01/2025', 'url': 'https://www.agenceecofin.com/actualites/2501-125238-gabon-la-production-industrielle-de-bois-en-hausse-de-4-au-3e-trimestre-2024'}

🔹 Test avec le modèle all-mpnet-base-v2
📄 alors que brice oligui n’guema 